In [1]:
%matplotlib inline
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy as sp
import scipy.io

In [2]:
from utils import *

In [4]:
cd real/

/home/spiaggesi/disene/exps/real


# DiSeNE 

In [5]:
#model:    "isgc" (DiSe-GAE), "imlp" (DiSe-FCAE)
#dataset:  "cora", "wiki", "facebook", "ppi"

In [6]:
!python exp_disene.py --runs=1 --model='isgc' --dataset='cora'

cora ISGC 128 128 0 embeddings saved
cora ISGC 128 128 0 dim. metrics saved
cora ISGC 128 128 0 pos. metrics saved


In [7]:
D_hid = 128
D_out = 128

In [9]:
emb_scores = np.load(f'../../output/real/isgc-walks-featureless-1hops-5window/linearshap_metrics/cora_ISGC_{D_hid}dims_{D_out}dims.0.scores.npz', allow_pickle=True)
emb_dist = np.load(f'../../output/real/isgc-walks-featureless-1hops-5window/linearshap_metrics/cora_ISGC_{D_hid}dims_{D_out}dims.0.spath.npz', allow_pickle=True)

In [10]:
Z = np.load(f'../../output/real/isgc-walks-featureless-1hops-5window/cora_ISGC_{D_hid}dims_{D_out}dims.0.npz', allow_pickle=True)['arr_0']
emb_masks = np.load(f'../../output/real/isgc-walks-featureless-1hops-5window/linearshap_metrics/cora_ISGC_{D_hid}dims_{D_out}dims.0.masks.npz', allow_pickle=True)['arr_0']

#### Comprehensibility and Sparsity

In [12]:
print('comprehensibility: ', round(emb_scores['f1_weighted'].max(axis=0).mean(), 4))
print('sparsity: ', round(1.-emb_scores['dim_ent_weighted'].mean(), 4))

comprehensibility:  0.8214
sparsity:  0.5041


#### Overlap consistency

In [13]:
from scipy.stats import pearsonr
from scipy.spatial.distance import pdist, squareform
def compute_correlation(x,y):
    if x.shape[0]<2:
        return 0.
    else:
        return np.nan_to_num(np.abs(pearsonr(x,y)[0]))
        
z_corr = np.nan_to_num(np.abs(1. - pdist(Z.T, 'correlation')))
print('overlap consistency: ', round(compute_correlation(z_corr[z_corr!=0.], emb_scores['jaccard_score'][z_corr!=0.]), 4))

overlap consistency:  0.9171


#### Positional Coherence

In [14]:
edge_idx = emb_scores['edge_index']
dims, edges = np.where(emb_masks>THRESH)
Z = Z[np.unique(edge_idx)][:, np.unique(dims)].T

In [15]:
d_corr = np.array([compute_correlation(Z[d], emb_dist['sum'][d]) for d in range(Z.shape[0])])
d_corr_rand = np.array([[compute_correlation(Z[d], emb_dist['sum'][j]) for d,j in zip(range(Z.shape[0]), np.random.permutation(range(Z.shape[0])))]
                       for _ in range(5)])
print('positional coherence: ', round(np.mean(d_corr)/np.mean(d_corr_rand+THRESH), 4))

positional coherence:  5.5147


# DeepWalk, InfWalk, GAE, GraphSAGE 

In [15]:
#model:    "deepwalk", "infwalk", "gae", "sage"
#dataset:  "cora", "wiki", "facebook", "ppi"

In [16]:
!python exp_baseline.py --runs=1 --model='gae' --dataset='cora'

cora GAE 2 0 embeddings saved
cora GAE 2 0 dim. metrics saved
cora GAE 2 0 pos. metrics saved
cora GAE 4 0 embeddings saved
cora GAE 4 0 dim. metrics saved
cora GAE 4 0 pos. metrics saved
cora GAE 8 0 embeddings saved
cora GAE 8 0 dim. metrics saved
cora GAE 8 0 pos. metrics saved
cora GAE 16 0 embeddings saved
cora GAE 16 0 dim. metrics saved
cora GAE 16 0 pos. metrics saved
cora GAE 32 0 embeddings saved
cora GAE 32 0 dim. metrics saved
cora GAE 32 0 pos. metrics saved
cora GAE 64 0 embeddings saved
cora GAE 64 0 dim. metrics saved
cora GAE 64 0 pos. metrics saved
cora GAE 128 0 embeddings saved
cora GAE 128 0 dim. metrics saved
cora GAE 128 0 pos. metrics saved
cora GAE 256 0 embeddings saved
cora GAE 256 0 dim. metrics saved
cora GAE 256 0 pos. metrics saved
cora GAE 512 0 embeddings saved
cora GAE 512 0 dim. metrics saved
cora GAE 512 0 pos. metrics saved


In [17]:
D_out = 128

In [18]:
emb_scores = np.load(f'../../output/real/gae-featureless/linearshap_metrics/cora_GAE_{D_out}dims.0.scores.npz', allow_pickle=True)
emb_dist = np.load(f'../../output/real/gae-featureless/linearshap_metrics/cora_GAE_{D_out}dims.0.spath.npz', allow_pickle=True)

In [19]:
Z = np.load(f'../../output/real/gae-featureless/cora_GAE_{D_out}dims.0.npz', allow_pickle=True)['arr_0']
emb_masks = np.load(f'../../output/real/gae-featureless/linearshap_metrics/cora_GAE_{D_out}dims.0.masks.npz', allow_pickle=True)['arr_0']

#### Comprehensibility and Sparsity

In [20]:
print('comprehensibility: ', round(emb_scores['f1_weighted'].max(axis=0).mean(), 4))
print('sparsity: ', round(1.-emb_scores['dim_ent_weighted'].mean(), 4))

comprehensibility:  0.2977
sparsity:  0.1789


#### Overlap consistency

In [21]:
from scipy.stats import pearsonr
from scipy.spatial.distance import pdist, squareform
def compute_correlation(x,y):
    if x.shape[0]<2:
        return 0.
    else:
        return np.nan_to_num(np.abs(pearsonr(x,y)[0]))
        
z_corr = np.nan_to_num(np.abs(1. - pdist(Z.T, 'correlation')))
print('overlap consistency: ', round(compute_correlation(z_corr[z_corr!=0.], emb_scores['jaccard_score'][z_corr!=0.]), 4))

overlap consistency:  0.229


#### Positional Coherence

In [22]:
edge_idx = emb_scores['edge_index']
dims, edges = np.where(emb_masks>THRESH)
Z = Z[np.unique(edge_idx)][:, np.unique(dims)].T

In [23]:
d_corr = np.array([compute_correlation(Z[d], emb_dist['sum'][d]) for d in range(Z.shape[0])])
d_corr_rand = np.array([[compute_correlation(Z[d], emb_dist['sum'][j]) for d,j in zip(range(Z.shape[0]), np.random.permutation(range(Z.shape[0])))]
                       for _ in range(5)])
print('positional coherence: ', round(np.mean(d_corr)/np.mean(d_corr_rand+THRESH), 4))

positional coherence:  1.023


#### Plausibility

In [25]:
print('plausibility: ', round(compute_plausibility_score(i_masks, i_scores, i_task), 4))

plausibility:  0.1608


# DeepWalk+DINE, GAE+DINE

In [26]:
#model:    "deepwalk", "gae"
#dataset:  "cora", "wiki", "facebook", "ppi"

In [25]:
!python exp_dine.py --runs=1 --model='gae' --dataset='cora'

cora GAE 8 128 0 dine embeddings saved
cora GAE 8 128 0 dim. metrics saved
cora GAE 8 128 0 pos. metrics saved
cora GAE 16 128 0 dine embeddings saved
cora GAE 16 128 0 dim. metrics saved
cora GAE 16 128 0 pos. metrics saved
cora GAE 32 128 0 dine embeddings saved
cora GAE 32 128 0 dim. metrics saved
cora GAE 32 128 0 pos. metrics saved
cora GAE 64 128 0 dine embeddings saved
cora GAE 64 128 0 dim. metrics saved
cora GAE 64 128 0 pos. metrics saved
cora GAE 128 128 0 dine embeddings saved
cora GAE 128 128 0 dim. metrics saved
cora GAE 128 128 0 pos. metrics saved
cora GAE 256 128 0 dine embeddings saved
cora GAE 256 128 0 dim. metrics saved
cora GAE 256 128 0 pos. metrics saved
cora GAE 512 128 0 dine embeddings saved
cora GAE 512 128 0 dim. metrics saved
cora GAE 512 128 0 pos. metrics saved


In [26]:
D_in = 128
D_out = 128

In [27]:
emb_scores = np.load(f'../../output/real/gae+dine-featureless/linearshap_metrics/cora_GAE_{D_in}dims.DINE_{D_out}dims.0.scores.npz', allow_pickle=True)
emb_dist = np.load(f'../../output/real/gae+dine-featureless/linearshap_metrics/cora_GAE_{D_in}dims.DINE_{D_out}dims.0.spath.npz', allow_pickle=True)

In [28]:
Z = np.load(f'../../output/real/gae+dine-featureless/cora_GAE_{D_in}dims.DINE_{D_out}dims.0.npz', allow_pickle=True)['arr_0']
emb_masks = np.load(f'../../output/real/gae+dine-featureless/linearshap_metrics/cora_GAE_{D_in}dims.DINE_{D_out}dims.0.masks.npz', allow_pickle=True)['arr_0']

#### Comprehensibility and Sparsity

In [29]:
print('comprehensibility: ', round(emb_scores['f1_weighted'].max(axis=0).mean(), 4))
print('sparsity: ', round(1.-emb_scores['dim_ent_weighted'].mean(), 4))

comprehensibility:  0.5568
sparsity:  0.2866


#### Overlap consistency

In [30]:
from scipy.stats import pearsonr
from scipy.spatial.distance import pdist, squareform
def compute_correlation(x,y):
    if x.shape[0]<2:
        return 0.
    else:
        return np.nan_to_num(np.abs(pearsonr(x,y)[0]))
        
z_corr = np.nan_to_num(np.abs(1. - pdist(Z.T, 'correlation')))
print('overlap consistency: ', round(compute_correlation(z_corr[z_corr!=0.], emb_scores['jaccard_score'][z_corr!=0.]), 4))

overlap consistency:  0.4304


#### Positional Coherence

In [31]:
edge_idx = emb_scores['edge_index']
dims, edges = np.where(emb_masks>THRESH)
Z = Z[np.unique(edge_idx)][:, np.unique(dims)].T

In [32]:
d_corr = np.array([compute_correlation(Z[d], emb_dist['sum'][d]) for d in range(Z.shape[0])])
d_corr_rand = np.array([[compute_correlation(Z[d], emb_dist['sum'][j]) for d,j in zip(range(Z.shape[0]), np.random.permutation(range(Z.shape[0])))]
                       for _ in range(5)])
print('positional coherence: ', round(np.mean(d_corr)/np.mean(d_corr_rand+THRESH), 4))

positional coherence:  2.3371
